**机器学习：将数据导入并分成测试集与训练集**

In [1]:
df=spark.sql("select * from user_erin.nbabetting1")

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,None,pyspark,idle,Link,,erin,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+-------+-------+-------+----+----------+-----+------+------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------+-------+
|      id|game_id|team_id|playoff| season|home|  realda

In [2]:
from pyspark.ml.feature import VectorAssembler
df_assembler1 = VectorAssembler(inputCols=['fg','fga','fg_pct','fg3','fg3a','fg3_pct','ft','fta','ft_pct','orb','drb','trb','ast','stl','blk','tov','pf','number_of_ots','attendance','win','newfg','newfga','newfg_pct','newfg3','newfg3a','newfg3_pct','newft','newfta','newft_pct','neworb','newdrb','newtrb','newast','newstl','newblk','newtov','newpf','newnumber_of_ots','newattendance','newwin','home','eelo','enewelo'],outputCol='features')
today= df_assembler1.transform(df)
today=today.select("game_id","curwin","features","year","season")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
dftrain=today.filter(today['season']!='2019-20')
dftest=today.filter(today['season']=='2019-20')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
today.select("season").distinct().show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+
| season|
+-------+
|2010-11|
|2012-13|
|2016-17|
|2009-10|
|2015-16|
|2014-15|
|2018-19|
|2019-20|
|2011-12|
|2017-18|
|2013-14|
+-------+

In [5]:
train=dftrain.filter(today['season']!='2018-19')
test=dftrain.filter(today['season']=='2018-19')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
train.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+--------------------+------+-------+
|game_id|curwin|            features|  year| season|
+-------+------+--------------------+------+-------+
|  51056|   1.0|[0.25674703911213...|2010.0|2009-10|
|  51418|   0.0|[-0.2464589158092...|2010.0|2009-10|
|  51428|   1.0|[0.25674703911214...|2010.0|2009-10|
|  51709|   0.0|[0.01712515581624...|2010.0|2009-10|
|  52001|   1.0|[-0.4381564224459...|2010.0|2010-11|
+-------+------+--------------------+------+-------+
only showing top 5 rows

***机器学习：LR模型的调参与预测***
- 用于分类的梯度提升决策树模型。该模型属于集成模型（Ensemble methods）家族。集成模型结合多个弱预测模型而形成一个强健的模型。
- 运用TrainValidationSplit（TVS）来进行参数调优。TVS对于每个参数组合只进行一次评估，当训练数据集足够大时，开销较小，并且产生的结果可靠。TVS会创建单个 (training, test) pair。它使用trainRatio参数将数据集split成两部分。例如： trainRatio=0.8，TVS会生成一个训练集（80%）和一个测试集（20%）。TVS最后会使用Estimator、以及最好的ParamMap，对整个数据集进行拟合。
- 运用param_grid方程来定义TVS检测的参数，最后一共12种参数组合模型被测试。
- regParam 正则项的参数，控制损失函数与惩罚项的比例，防止整个训练过程过拟合
- maxIter 设置的越大结果越精确，但是设置的越大也就意味着越耗时；一般建议10-20。
- elasticNetParam 使用L1范数还是L2范数,设置为0.0为L2正则化，设置为1.0为L1正则化，设置为（0.0，1.0）为L1,L2组合

In [7]:
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator
import pyspark.sql.functions as f
lr = LogisticRegression(featuresCol = 'features',labelCol = 'curwin')
evaluator  = MulticlassClassificationEvaluator(labelCol="curwin", predictionCol="prediction",metricName='accuracy')
param_grid = ParamGridBuilder() \
            .addGrid(lr.regParam, [0.1, 0.01]) \
            .addGrid(lr.maxIter, [10, 20]) \
            .addGrid(lr.elasticNetParam,[0.0,0.5,1.0]) \
            .build()


tvs = TrainValidationSplit(estimator=lr,
                           estimatorParamMaps=param_grid,
                           evaluator=evaluator,
                           trainRatio = 0.8)

model2=tvs.fit(train)
model2=model2.bestModel
prediction=model2.transform(test)
acc=evaluator.evaluate(prediction)
print(acc)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.6592987804878049

**最佳模型表现**

In [8]:
lrprediction1=model2.transform(dftest)
print("lr_trained_model accuracy for nba game prediction for 2019-20",evaluator.evaluate(lrprediction1))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lr_trained_model accuracy for nba game prediction for 2019-20 0.6711165048543689

In [9]:
lrprediction1.show(20)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+--------------------+------+-------+--------------------+--------------------+----------+
|game_id|curwin|            features|  year| season|       rawPrediction|         probability|prediction|
+-------+------+--------------------+------+-------+--------------------+--------------------+----------+
|  63676|   1.0|[1.35900770227325...|2019.0|2019-20|[-1.9676769569599...|[0.12263862347943...|       1.0|
|  63710|   0.0|[1.59862958556915...|2019.0|2019-20|[-0.6410280034131...|[0.34501419480092...|       1.0|
|  63448|   0.0|[-0.5579673640939...|2019.0|2019-20|[-0.3034372659205...|[0.42471743074319...|       1.0|
|  63461|   1.0|[-0.1506101624908...|2019.0|2019-20|[-0.4081388663797...|[0.39935846992663...|       1.0|
|  63479|   1.0|[0.08901172080501...|2019.0|2019-20|[-0.0477740547793...|[0.48805875740362...|       1.0|
|  63498|   1.0|[-0.1266479741612...|2019.0|2019-20|[-0.7304025213971...|[0.32510640308370...|       1.0|
|  63586|   1.0|[0.20882266245296...|2019.0|20